# 基于Colab和GD可以快速搭建stable-diffusion服务的方案
# <font size="3"> 英文原方案来自于 https://github.com/TheLastBen/fast-stable-diffusion<br/>中文编辑：春春<br/>原作者支持.[Support](https://ko-fi.com/thelastben)</font>

In [ ]:
#@markdown # 1.查看英伟达显卡信息
!nvidia-smi

In [ ]:
#@markdown # 2.挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown # 3.安装/升级 AUTOMATIC1111
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # 4.requirements
#@markdown # <font size="3"> 是第一次运行，此条运行完后请点击第5步<br/>如果不是第一次运行，此条运行完后请点击第7步</font>

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf A1111_dep.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # 5.默认Model下载和安装
Model_Version = "V2.1-768px" #@param [ "1.5", "v1.5 Inpainting ", "V2.1-512px", "V2.1-768px"]

Redownload_the_original_model = False #@param {type:"boolean"}

def rmv():
  !wget -q -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/README.md
  !mv /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
  time.sleep(2)
  !rm /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
  clear_output()

if Redownload_the_original_model:
  with capture.capture_output() as cap:
    rmv()

#@markdown Or
Path_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full path of your trained model or to a folder containing multiple models.

#@markdown Or
MODEL_LINK = "" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}
Use_temp_storage = False #@param {type:"boolean"}



def newmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-v1-5"
      if os.path.exists('/content/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
        !wget -q -O stable-diffusion-v1-5/vae/diffusion_pytorch_model.bin https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-v1-5"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V1.5','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-v1-5'):
      !rm -r /content/stable-diffusion-v1-5

    return model

def V2():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !mkdir "/content/stable-diffusion-V2"
      %cd "/content/stable-diffusion-V2"
      !git init
      !git lfs install --system --skip-repo
      if Model_Version == "V2.1-768px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
      elif Model_Version == "V2.1-512px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
      !git config core.sparsecheckout true
      !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
      !git pull origin fp16
      %cd /content
      !wget -O convertosdv2.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosdv2.py
      clear_output()
      !python /content/convertosdv2.py --fp16 /content/stable-diffusion-V2 /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
      !rm /content/convertosdv2.py
      if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
        model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
        clear_output()
        inf('\u2714 Done','success', '50px')
      else:
        inf('\u2718 Something went wrong, try again','danger', "250px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V2','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git
      !rm -r /content/convertosdv2.py
    if os.path.exists('/content/stable-diffusion-V2'):
      !rm -r /content/stable-diffusion-V2

    return model

def inpmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-inpainting"
      if os.path.exists('/content/stable-diffusion-inpainting'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /content/stable-diffusion-inpainting/vae
        !mv /content/sd-vae-ft-mse /content/stable-diffusion-inpainting/vae        
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-inpainting"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")


    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
      clear_output()
      inf('\u2714 inpainting model already exists','primary', '250px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-inpainting'):
      !rm -r /content/stable-diffusion-inpainting

    return model

if (Path_to_MODEL !=''):
  if os.path.exists(str(Path_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(Path_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        Path_to_MODEL=input()
      if os.path.exists(str(Path_to_MODEL)):
        inf('\u2714 Using the trained model.','success', '200px')

  model=Path_to_MODEL

elif MODEL_LINK != "":
  gdrv="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
  tmp="/content"
  pth=tmp if Use_temp_storage else gdrv
  %cd $pth
  clear_output()
  if not safetensors:
    modelname="model.ckpt"
  else:
    modelname="model.safetensors"
  !gdown --fuzzy -O $modelname $MODEL_LINK
  if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
      model=f'{pth}/{modelname}'
      clear_output()
      inf('\u2714 Model downloaded, using the trained model.','success', '350px')
  else:
    if Use_temp_storage:
      !rm $pth/$modelname
    else:
      rmv()
    inf('\u2718 Wrong link, check that the link is valid','danger', "300px")


elif Model_Version=="1.5":
  model=newmdl()

elif Model_Version=="V2.1-512px" or Model_Version=="V2.1-768px":
  model=V2()

else:
   model=inpmdl()

In [ ]:
#@markdown # 6.下载安装ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse

Model = "All" #@param [ "All", "Canny", "Depth", "HED", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "T2iadapter_Models"]

#@markdown - Download/update ControlNet extension and its models.

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
Depth='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
HED='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
MLSD='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
Normal='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
OpenPose='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
Scribble='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
Seg='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'


with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git pull
    %cd /content

!cp /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/*.yaml /content/gdrive/MyDrive/sd/stable-diffusion-webui/models
mdldir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models"

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt
with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()
!rm CN_models.txt

if Model == "All": 
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

else:
  download(globals()[Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # 7.启动Stable-Diffusion
from IPython.utils import capture
import time
import sys
import fileinput

Use_localtunnel = False #@param {type:"boolean"}

User = "admin" #@param {type:"string"}
Password= "123456" #@param {type:"string"}
#@markdown - Add credentials to your Gradio interface (optional).

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.8/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars

[AddNet] Updating model hashes...
100% 3/3 [00:00<00:00, 149.23it/s]
[AddNet] Updating model hashes...
100% 3/3 [00:00<00:00, 653.25it/s]
Loading weights [44f90a0972] from /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/protogenX34Photorealism_1.safetensors
Creating model from config: /content/gdrive/MyDrive/sd/stable-diffusion-webui/configs/v1-inference.yaml
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Applying xformers cross attention optimization.
Textual inversion embeddings loaded(0): 
Model loaded in 50.4s (load weights from disk: 44.1s, create model: 1.4s, apply weights to model: 2.9s, apply half(): 1.2s, move model to device: 0.8s).
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b274ec4e-f6b1-4a52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
Startup time: 85.0s (import gradio: 3.9

In [ ]:
#@markdown # 下载工具
from tqdm import tqdm
import requests
import os
url = "https://imagecache.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/757d1bcb-b870-45b8-0bed-ce85568a2400/width=400/255442"  #@param {type:"string"}
destPath = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/dalcefoPainting_v4.png"  #@param {type:"string"}
with requests.get(url, stream=True) as r:
    length = int(r.headers['Content-Length'])
    print("Total:",length)
    r.raise_for_status()
    with open(destPath, 'wb') as f:
        pbar = tqdm(total=length)
        for chunk in r.iter_content(chunk_size=8192):
            if chunk: 
                f.write(chunk)
                pbar.update(len(chunk))

file_size = os.path.getsize(destPath)
if file_size == length:
    print("\nDone")
else:
    raise RuntimeError("File size mismatch: got {} of {}".format(file_size, length))

In [ ]:
#@markdown # 批量JPG转PNG
directory = '/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora' #@param {type:"string"}

files = os.listdir(directory)


# Then you rename the files 
for file_name in files:
    # You give the full path of the file
    old_name = os.path.join(directory, file_name)

    # You CHANGE the extension
    new_name = old_name.replace('.jpg', '.png')
    os.rename(old_name, new_name)

In [ ]:
#@markdown # 设置 requirements.txt 
%%writefile requirements.txt 
blendmodes 
accelerate 
basicsr 
fonts 
font-roboto 
gfpgan 
gradio==3.16.2 
invisible-watermark 
numpy 
omegaconf 
opencv-contrib-python 
requests 
piexif 
Pillow 
pytorch_lightning==1.7.7 
realesrgan 
scikit-image>=0.19 
timm==0.4.12 
transformers==4.25.1 
torch 
einops 
jsonmerge 
clean-fid 
resize-right 
torchdiffeq 
kornia 
lark 
inflection 
GitPython 
torchsde 
safetensors 
psutil 
albumentations==0.4.3 
opencv-python 
pudb==2019.2 
imageio==2.9.0 
omegaconf==2.1.1 
test-tube>=0.7.5 
webdataset==0.2.5 
open-clip-torch==2.7.0 
kornia==0.6 
invisible-watermark>=0.1.5 
streamlit-drawable-canvas==0.8.0 
xformers 
fastapi

Overwriting requirements.txt


In [ ]:
#@markdown # 重新安装 requirements和fastapi 
!pip install -r requirements.txt 
!pip install git+https://github.com/openai/CLIP.git 
!pip install --upgrade fastapi==0.90.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 KB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 KB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 KB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━